In [ ]:
import numpy as np
import pandas as pd
import itertools
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from sklearn.feature_extraction.text import TfidfVectorizer
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.rdMolDescriptors import CalcNumRings, CalcNumAromaticRings
import pywt


def extract_features_protein(seq):
    features = {}

    # Biological features
    features['amino_acid_composition'] = np.array(
        [seq.count(aa) for aa in 'ACDEFGHIKLMNPQRSTVWY'])
    pa = ProteinAnalysis(seq)
    features['molecular_weight'] = pa.molecular_weight()
    features['isoelectric_point'] = pa.isoelectric_point()
    features['aromaticity'] = pa.aromaticity()
    features['instability_index'] = pa.instability_index()
    features['gravy'] = pa.gravy()
    features['secondary_structure_fraction'] = pa.secondary_structure_fraction()

    # phsio
    features['logp'] = logp(seq)
    features['tpsa'] = tpsa(seq)
    features['rotatable_bonds'] = rotatable_bonds(seq)

    # Computational features
    features['sequence_length'] = len(seq)
    features['sequence_complexity'] = shannon_entropy(seq)
    features['kmer_frequencies'] = np.array([seq.count(kmer) for kmer in [
                                            'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']])
    features['auto_correlation'] = moran_i(seq)
    features['pseaac'] = pseaac(seq)
    features['conjoint_triad'] = conjoint_triad(seq)
    features['quasi_sequence_order'] = quasi_sequence_order(seq)
    features['wavelet_features'] = wavelet_features(seq)

    # rdkit_features

    from rdkit import Chem
    from rdkit.Chem import Descriptors
    from rdkit.Chem.rdMolDescriptors import CalcNumRings, CalcNumAromaticRings
    import itertools

    mol = Chem.MolFromSequence(seq)

    features['molecular_weight'] = Descriptors.MolWt(mol)
    features['heavy_atom_count'] = Descriptors.HeavyAtomCount(mol)
    features['h_bond_acceptors'] = Descriptors.NumHAcceptors(mol)
    features['h_bond_donors'] = Descriptors.NumHDonors(mol)
    features['ring_count'] = CalcNumRings(mol)
    features['aromatic_ring_count'] = CalcNumAromaticRings(mol)

    # Custom ring count calculation
    ri = mol.GetRingInfo()
    aliphatic_rings = 0
    saturated_rings = 0
    unsaturated_rings = 0
    for ring in ri.AtomRings():
        ring_atoms = [mol.GetAtomWithIdx(i) for i in ring]
        ring_bonds = [mol.GetBondBetweenAtoms(
            i, j) for i, j in itertools.combinations(ring, 2)]
        is_aromatic = any(b.GetIsAromatic()
                          for b in ring_bonds if b is not None)
        if is_aromatic:
            continue  # already counted as aromatic
        is_saturated = all(b.GetBondType(
        ) == Chem.rdchem.BondType.SINGLE for b in ring_bonds if b is not None)
        if is_saturated:
            saturated_rings += 1
        else:
            unsaturated_rings += 1
        aliphatic_rings += 1

    features['aliphatic_ring_count'] = aliphatic_rings
    features['saturated_ring_count'] = saturated_rings
    features['unsaturated_ring_count'] = unsaturated_rings
    # Custom ring count calculation

    return features


def shannon_entropy(seq):
    # Calculate Shannon entropy
    freqs = [seq.count(aa) / len(seq) for aa in 'ACDEFGHIKLMNPQRSTVWY']
    # add a small value to avoid zero frequencies
    freqs = [f + 1e-6 for f in freqs]
    return -sum([freq * np.log2(freq) for freq in freqs])


def logp(seq):
    mol = Chem.MolFromSequence(seq)
    logp_value = Descriptors.MolLogP(mol)
    return logp_value


def tpsa(seq):
    mol = Chem.MolFromSequence(seq)
    tpsa_value = Descriptors.TPSA(mol)
    return tpsa_value


def rotatable_bonds(seq):
    mol = Chem.MolFromSequence(seq)
    rotatable_bonds_value = Descriptors.NumRotatableBonds(mol)
    return rotatable_bonds_value


def moran_i(seq):
    # Calculate Moran's I
    freqs = [seq.count(aa) / len(seq) for aa in 'ACDEFGHIKLMNPQRSTVWY']
    return np.corrcoef(freqs[:-1], freqs[1:])[0, 1]


def pseaac(seq):
    # Calculate PseAAC
    pseaac_features = []
    for i in range(len(seq) - 2):
        pseaac_features.append(
            np.array([seq[i:i+3].count(aa) for aa in 'ACDEFGHIKLMNPQRSTVWY']))
    return np.array(pseaac_features)


def conjoint_triad(seq):
    # Calculate conjoint triad
    conjoint_triad_features = []
    for i in range(len(seq) - 2):
        conjoint_triad_features.append(
            np.array([seq[i:i+3].count(aa) for aa in 'ACDEFGHIKLMNPQRSTVWY']))
    return np.array(conjoint_triad_features)


def quasi_sequence_order(seq):
    # Calculate quasi-sequence order
    quasi_sequence_order_features = []
    for i in range(len(seq) - 2):
        quasi_sequence_order_features.append(
            np.array([seq[i:i+3].count(aa) for aa in 'ACDEFGHIKLMNPQRSTVWY']))
    return np.array(quasi_sequence_order_features)


def wavelet_features(seq):
    # Calculate wavelet features
    freqs = [seq.count(aa) / len(seq) for aa in 'ACDEFGHIKLMNPQRSTVWY']
    coeffs = pywt.wavedec(freqs, 'haar', level=2)
    return coeffs


# Example usage
seq = 'MAEGEITTQPVLQAGVLQAGVLQAGVLQAGVLQAGVLQAGVLQAGVL'
features = extract_features_protein(seq)
# df = pd.DataFrame(features)
print(features)

In [ ]:
All_lines = sequencesMP + sequencesNonMP


def calculate_features_proteins(sequences):
    i = 0
    features = []
    for seq in sequences:
        print(i)
        i = i+1
        feature = extract_features_protein(seq)
        features.append(feature)
    return features


sequences = All_lines
features = calculate_features_proteins(sequences)

df = pd.DataFrame(features)
print(df)


In [ ]:
numerical_features = ["molecular_weight", "isoelectric_point", "aromaticity",
                      "instability_index", "gravy", 'sequence_length', 'sequence_complexity', 'auto_correlation']
df[numerical_features].to_csv('Top8clean.tsv', sep='\t', index=False)